In [256]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [42]:
base = 'http://mit.spbau.ru'

In [6]:
r = requests.get('http://mit.spbau.ru/students/se')
html = r.text

In [8]:
soup = BeautifulSoup(html, 'html.parser')

In [90]:
names = []
images = []
hrefs = []
for div in soup.find_all('div', {'class': 'field-content alumni-userpic'}):
    for a in div.find_all('a'):
        names.append(a.text)
    for img in div.find_all('img'):
        images.append(img['src'])
        hrefs.append(base + a['href'])

In [108]:
df = pd.DataFrame({'name': [name for name in names if 0 < len(name.split()) < 5], \
                   'image': images, 'url': hrefs})

In [234]:
fields = ['field field-name-field-program field-type-list-text field-label-above',
         'field field-name-field-gradyead field-type-list-integer field-label-inline clearfix',
         'field field-name-field-thesistopic field-type-text field-label-above',
         'field field-name-field-advisor field-type-text field-label-above', 
         'field field-name-field-wherenow field-type-text field-label-above',
         'field field-name-field-before-au field-type-text field-label-inline clearfix']
image_class = 'field field-name-field-alumni-photo field-type-image field-label-hidden'

In [237]:
blocks = []
images = []
for url in df.url:
    try:
        block = []
        r = requests.get(url)
        html = r.text
        soup = BeautifulSoup(html, 'html.parser')
        for field in fields:
            try:
                block.append(soup.find('div', {'class': field}).text)
            except:
                continue
        blocks.append(block)
        images.append(soup.find('div', {'class': image_class}).find('img')['src'])
    except:
        blocks.append([])
        images.append('None')
        print(url)

http://mit.spbau.ruhttp://bioinf.spbau.ru/sites/default/files/AlexeyGurevich_thesis_fulltext.pdf
http://mit.spbau.ruhttp://bioinf.spbau.ru/sites/default/files/AndreyPrjibelski_thesis_fulltext.pdf


In [239]:
def get_info(block):
    dicts = [{k: v} for k, v in [line.split(':') for line in block]]
    info = dict(pair for d in dicts for pair in d.items())
    return info

In [240]:
infos = []
for block in blocks:
    infos.append(get_info(block))

df['info'] = infos

In [242]:
df['big_images'] = images

In [261]:
df.head()

,name,image,url,info,big_images
0,Абрамов Иван Александрович,http://mit.spbau.ru/files/alumni/%D0%90%D0%B1%...,http://mit.spbau.ru/node/906,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/%D0%90%D0%B1%...
1,Акимов Евгений Владимирович,http://mit.spbau.ru/files/alumni/%D0%90%D0%BA%...,http://mit.spbau.ru/node/911,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/%D0%90%D0%BA%...
2,Бибаев Виталий Игоревич,http://mit.spbau.ru/files/alumni/bibaevsm.jpg,http://mit.spbau.ru/node/914,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/bibaev.jpg
3,Васильев Роман Алексеевич,http://mit.spbau.ru/files/alumni/roman.vasilie...,http://mit.spbau.ru/node/907,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/roman.vasilie...
4,Гарифуллин Шамиль Раифович,http://mit.spbau.ru/files/alumni/%D0%93%D0%B0%...,http://mit.spbau.ru/node/901,"{'Программа обучения': ' SE', 'Год окончания':...",http://mit.spbau.ru/files/alumni/%D0%93%D0%B0%...


In [266]:
path = 'au_photos'
for idx in range(len(df)):
    if df.big_images[idx] != 'None':
        img_data = requests.get(df.big_images[idx]).content
    else:
        img_data = requests.get(df.image[idx]).content
    with open(os.path.join(path, f'{df.name[idx].split()[0]}.png'), 'wb') as handler:
        handler.write(img_data)